In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os

from fastai.text import *
from fastai import *

In [3]:
path = Path('/home/jupyter/insight_project/Project-M/data/preprocessed/csv')

In [4]:
filenames = path.ls()
filenames.remove(Path('/home/jupyter/insight_project/Project-M/data/preprocessed/csv/models'))
len(filenames)

71

In [5]:
df_1 = pd.read_csv(filenames[0])
df_1.shape

(101, 3)

In [6]:
df_1['Label'].value_counts()

1    79
0    22
Name: Label, dtype: int64

In [7]:
model = get_language_model(AWD_LSTM,30000)

In [22]:
model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(30000, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(30000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=30000, bias=True)
    (output_dp): RNNDropout()
  )
)

In [8]:
data_lm = load_data(path/'models','data_lm.pkl',bs=64)

In [8]:
learn = language_model_learner(data_lm,AWD_LSTM,drop_mult=0.3)

In [8]:
!ls {path}/'models'

1a.pth			    fine-tuned-enc-fwd.pth  textlist_class_seed42
awd_lstm_baseline-0917.pth  fine-tuned-fwd.pth	    tmp.pth
data_lm.pkl		    models


In [8]:
learn.load(path/'models/fine-tuned-fwd');

In [10]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(35584, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(35584, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=35584, bias=True)
    (output_dp): RNNDropout()
  )
)

In [11]:
model_head = learn.model[0]

In [12]:
model_head

AWD_LSTM(
  (encoder): Embedding(35584, 400, padding_idx=1)
  (encoder_dp): EmbeddingDropout(
    (emb): Embedding(35584, 400, padding_idx=1)
  )
  (rnns): ModuleList(
    (0): WeightDropout(
      (module): LSTM(400, 1152, batch_first=True)
    )
    (1): WeightDropout(
      (module): LSTM(1152, 1152, batch_first=True)
    )
    (2): WeightDropout(
      (module): LSTM(1152, 400, batch_first=True)
    )
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
    (2): RNNDropout()
  )
)

In [13]:
x,y = next(iter(data_lm.train_dl))

In [14]:
x.shape,y.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [15]:
preds = model_head(x)

In [16]:
len(preds[1])

3

In [83]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(35584, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(35584, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=35584, bias=True)
    (output_dp): RNNDropout()
  )
)

In [87]:
preds[0][0].shape,preds[0][1].shape,preds[0][2].shape

(torch.Size([64, 70, 1152]),
 torch.Size([64, 70, 1152]),
 torch.Size([64, 70, 400]))

In [88]:
len(preds)

2

In [90]:
preds[1][0].shape,preds[1][1].shape,preds[1][2].shape

(torch.Size([64, 70, 1152]),
 torch.Size([64, 70, 1152]),
 torch.Size([64, 70, 400]))

In [91]:
preds_full = learn.model(x)

In [92]:
len(preds_full)

3

In [93]:
out = preds[1][2]

In [94]:
out.shape

torch.Size([64, 70, 400])

In [96]:
out[0].shape

torch.Size([70, 400])

In [9]:
data_siamese = (TextList
        .from_df(df=df_1,path=path,cols='Body',vocab=data_lm.vocab)
        .split_by_rand_pct(0.3,seed=42)
        .label_from_df(cols='Label')
       )

In [10]:
import random
from IPython.core.debugger import Tracer

In [11]:
data_siamese.train[3][0]

Text xxbos xxmaj date idea # xxunk : minecraft

In [12]:
class SiameseDataset(Dataset):
    def __init__(self,ds):
        self.ds = ds
        self.labels = ds.y #labels
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self,i):
        same = lambda : random.randint(0,1) #construct same pair or different pair
        return self.siameseItem(i,same())
    
    def siameseItem(self,i,same):
        label = self.ds.inner_df.iloc[i]['Label']
        if same:
            same_label_list = np.where(self.ds.inner_df['Label'] == label)[0]
            if len(same_label_list) > 1: #have more than 1 items, then don't pick itself
                same_label_list = np.delete(same_label_list,np.argwhere(same_label_list==i))
            msg_index = same_label_list[np.random.randint(0,len(same_label_list))] #pick another msg
            #print(msg_index,len(same_label_list))
        else:
            #print(1111)
            diff_label_list = np.where(self.ds.inner_df['Label'] != label)[0]
            msg_index = diff_label_list[np.random.randint(0,len(diff_label_list))]
            #print(msg_index,len(diff_label_list),1)
        return self.constructSiamese(i,msg_index,same)
    
    def constructSiamese(self,msg1_index,msg2_index,same):
        item_1 = self.ds[msg1_index][0]
        item_2 = self.ds[msg2_index][0]
        return ([item_1,item_2],same)

In [13]:
train_dl = DataLoader(
    dataset = SiameseDataset(data_siamese.train),
    batch_size = 16,
    shuffle = True,
    num_workers = 0
)

valid_dl = DataLoader(
    dataset = SiameseDataset(data_siamese.valid),
    batch_size = 16,
    num_workers = 0
)

In [16]:
data_siamese = TextClasDataBunch(train_dl,valid_dl)

In [17]:
x,y = next(iter(data_siamese.train_dl))

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 12 and 37 in dimension 1 at /tmp/pip-req-build-58y_cjjl/aten/src/TH/generic/THTensor.cpp:689

In [58]:
_same_label_list = np.where(data_siamese.train.inner_df['Label'] != 1)[0]
_same_label_list

array([ 2,  4,  6, 11, 13, 23, 24, 31, 35, 38, 42, 43, 49, 52, 53, 63, 64, 69])

In [56]:
_same_label_list = np.delete(_same_label_list,np.argwhere(_same_label_list==3))
_same_label_list

array([ 0,  1,  5,  7, ..., 66, 67, 68, 70])

In [59]:
_same_label_list[np.random.randint(0,len(_same_label_list))]

64

In [69]:
item_1 = data_siamese.train[3]
item_1

(Text xxbos xxmaj date idea # xxunk : minecraft, Category 1)

In [38]:
item_2 = data_siamese.train[64]
item_2

(Text xxbos 33 / 90 xxunk just thirty three days to the best date of the year ! i love you infinitely , thank you for existing . ♡,
 Category 0)

In [23]:
%debug

> /opt/anaconda3/lib/python3.7/site-packages/torch/_utils.py(369)reraise()
    365             # KeyError calls repr() on its argument (usually a dict key). This
    366             # makes stack traces unreadable. It will not be changed in Python
    367             # (https://bugs.python.org/issue2651), so we work around it.
    368             msg = KeyErrorMessage(msg)
--> 369         raise self.exc_type(msg)

ipdb> u
> /opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py(846)_process_data()
    844         self._try_put_index()
    845         if isinstance(data, ExceptionWrapper):
--> 846             data.reraise()
    847         return data
    848 

ipdb> u
> /opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py(819)__next__()
    817             else:
    818                 del self.task_info[idx]
--> 819                 return self._process_data(data)
    820 
    821     next = __next__  # Python 2 compatibility

ipdb> u
> /opt/anac